In [1]:
#Module configuration
#from py2neo import Graph #Database connection
import neomodel
import pandas as pd
import datetime

In [2]:
from neomodel import config

neo4jServerAdress = "172.19.242.116"
neo4jConnectionPort = 7687  #Connection to server bolt
user= "neo4j"
password= "Mypassword%2023"
config.DATABASE_URL='bolt://' +user + ':' + password + '@' + neo4jServerAdress + ':' + str(neo4jConnectionPort)

In [3]:
from neomodel import (config, StructuredNode, StringProperty, IntegerProperty,DateTimeProperty,
                    DateTimeFormatProperty,UniqueIdProperty, RelationshipTo,StructuredRel)

config.DATABASE_URL='bolt://' +user + ':' + password + '@' + neo4jServerAdress + ':' + str(neo4jConnectionPort)


#NIP	J_Parcours	DateDebutActe	DateFinActe	UF	Service	Phase_Parcours	Activite	Dimension_Parcours	Ref_Acte	id_Sequence	Type_Sequence	id_sejour

class FOLLOWED_BY(StructuredRel):
    Nb_Days = IntegerProperty(index=True, default=0)
    #NIP=StringProperty(required=True)

class Acte(StructuredNode):
    Ref_Acte = StringProperty(required=True)
    J_Parcours=IntegerProperty(required=True)
    Duree = IntegerProperty(required=True)
    #DateDebutActe = DateTimeFormatProperty(required=True)  #Would be better to use DateTimeFormatProperty
    #DateFinActe = DateTimeFormatProperty(required=True)  #Would be better to use DateTimeFormatProperty
    UF = StringProperty(required=True)
    Service = StringProperty(required=True)
    Phase_Parcours =StringProperty(required=True)
    Activite =StringProperty(required=True)
    Dimension_Parcours = StringProperty(required=True)
    Sejour_id = StringProperty(required=True)
    Sequence_id = StringProperty(required=True)
    Sequence_Type = StringProperty(required=True)
    
    Carepath = RelationshipTo('Acte', 'FOLLOWED_BY', model=FOLLOWED_BY)

class Patient(StructuredNode):
    NIP = StringProperty(unique_index=True, required=True)
    J0 = StringProperty(required=True)
    J_Parcours = IntegerProperty(required=False, default=0)
    Carepath = RelationshipTo('Acte', 'BEGIN_CAREPATH_WITH', model=FOLLOWED_BY)



In [4]:
# Define the filepath of the import csvfile
file_path = r'..\01_Datasets\DL_Project_Dataset_Sample_V5.csv'
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, sep=';')
# Display the first few rows of the DataFrame
df


,NIP,J_Parcours,DateDebutActe,DateFinActe,UF,Service,Phase_Parcours,Activite,Dimension_Parcours,Ref_Acte,id_Sequence,Type_Sequence,id_sejour
0,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,8532,Médecine Nucléaire,Diagnostic,TEP,Soins,FTNTP2,12513,INIT_NEW,N1771659
1,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,8532,Médecine Nucléaire,Diagnostic,TEP,Soins,ZZQL016,12513,INIT_NEW,N1771659
2,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,1001,Médecine Nucléaire,Diagnostic,TEP,Soins,FTNTP2,12513,INIT_NEW,N160017801
3,N201900002,37,2019-02-08 00:00:00.000,2019-02-08 00:00:00.000,9050,Radiothérapie,Consultation,Consultations,Soins,CS,12513,INIT_NEW,N1791270
4,N201900002,37,2019-02-08 00:00:00.000,2019-02-08 00:00:00.000,9050,Radiothérapie,Consultation,Consultations,Soins,MCS,12513,INIT_NEW,N1791270
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,N201900413,704,2020-12-08 00:00:00.000,2020-12-08 00:00:00.000,1933,Laboratoire,Diagnostic,Diagnostic,Soins,1609,12809,SUIVI_LT,N2134149
1907,N201900413,704,2020-12-08 00:00:00.000,2020-12-08 00:00:00.000,1933,Laboratoire,Diagnostic,Diagnostic,Soins,514,12809,SUIVI_LT,N2134149
1908,N201900413,704,2020-12-08 00:00:00.000,2020-12-08 00:00:00.000,1933,Laboratoire,Diagnostic,Diagnostic,Soins,9005,12809,SUIVI_LT,N2134149
1909,N201900413,704,2020-12-08 00:00:00.000,2020-12-08 00:00:00.000,1933,Laboratoire,Diagnostic,Diagnostic,Soins,522,12809,SUIVI_LT,N2134149


In [5]:
#Function to extract information from a row
date_format = '%Y-%m-%d %H:%M:%S.%f'

def extract_data(row):
    from datetime import datetime
    acte = {
        'Jp': row.get('J_Parcours', None),
        'DDA': datetime.strptime(row.get('DateDebutActe', None), date_format),
        'DDF': datetime.strptime(row.get('DateFinActe', None), date_format),
        'Ref': row.get('Ref_Acte', None),
        'NIP': row.get('NIP', None),
        'UFname': row.get('UF', None),
        'service': row.get('Service', None),
        'phase_parcours': row.get('Phase_Parcours', None).replace(" ", ""),
        'activite': row.get('Activite', None),
        'dim_parcours': row.get('Dimension_Parcours', None),
        'Sequence_id': row.get('id_Sequence', None),
        'Sequence_Type': row.get('Type_Sequence', None),
        'Sejour_id': row.get('id_sejour', None)
    }

    return acte

In [6]:
#Define a query function
def myquery(query, params=None):
    from neomodel import db
    results, meta = db.cypher_query(query, params, resolve_objects=True)
    
    
    return results, meta

In [7]:
#Define a function to create a new acte 
def create_acte(myActe):
    new_acte=Acte(
                    Ref_Acte = myActe['Ref'],
                    Duree = int((myActe['DDF'] - myActe['DDA']).total_seconds()),
                    J_Parcours = myActe['Jp'],
                    #DateDebutActe = myActe['DDA'],
                    #DateFinActe = myActe['DDF'] ,
                    UF = myActe['UFname'],
                    Service = myActe['service'],
                    Phase_Parcours = myActe['phase_parcours'],
                    Activite = myActe['activite'],
                    Dimension_Parcours = myActe['dim_parcours'],
                    Sejour_id = myActe['Sejour_id'],
                    Sequence_id = myActe['Sequence_id'],
                    Sequence_Type = myActe['Sequence_Type']     
                ).save()
    return new_acte

def create_Startnode(myActe):
    startNode=Patient(
                    NIP = myActe['NIP'],
                    J0 = myActe['DDA']
                    ).save()
    return startNode

In [8]:
#MODELIZATION 5 : 
#NOEUDS : ACTES , DIFFERENTIATED BY PHASE PARCOURS.
#RELATIONS : 
    # START : for the first acte
    # ACTES REALIZED BY DAY

#INITIALISATION   #CLEAR THE DATABASE
query="MATCH (n) DETACH DELETE n"
myquery(query)

Start=True
output=True
Previous_Nodes=[]
Active_Nodes=[]

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Extract values from the DataFrame
    myActe = extract_data(row)
    
    #Create a new acte
    new_acte=create_acte(myActe)

    if Start:
        #Create a start node
        startNode=create_Startnode(myActe)

        #LINK THE START NODE TO THE ACTES
        startNode.Carepath.connect(new_acte)
        Start=False

        #SET THE START NODE AS THE PREVIOUS NODE
        Previous_Nodes=[startNode]

    else:
        if oldActe['NIP']!=myActe['NIP']: #Check if we are dealing with the same patient
            Previous_Nodes=[]
            Active_Nodes=[new_acte]
            
            #Create a start node
            startNode=create_Startnode(myActe)

            #LINK THE START NODE TO THE ACTES
            startNode.Carepath.connect(new_acte)
            #SET THE START NODE AS THE PREVIOUS NODE
            Previous_Nodes=[startNode]

        else:            
            #Check if we have to change the previous node
            if myActe['Jp']!=oldActe['Jp']:
                Previous_Nodes=Active_Nodes
                Active_Nodes=[new_acte]

            else:
                Active_Nodes.append(new_acte)

            #LINK THE ACTE TO THE PREVIOUS ONE
            for node in Previous_Nodes:
                node.Carepath.connect(new_acte, {'Nb_Days': myActe['Jp']-node.J_Parcours})
            
    #Store the acte in a variable oldActe
    oldActe=myActe


In [9]:
#List exsiting graph in the database
from neomodel import db
query = """
CALL gds.graph.list()
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
ORDER BY graphName ASC
"""
results = db.cypher_query(query)
results

([['Mygraph', 1911, 5417], ['Mygraph3', 1911, 5417]],
 ['graphName', 'nodeCount', 'relationshipCount'])

In [10]:

#delete all existings graphs
for graph in results[0]:
    query = f"""
            CALL gds.graph.drop('{graph[0]}') YIELD graphName;
            """
    results = db.cypher_query(query)

In [11]:
#Node Embedings

#Create an in-Memory graph:
query = """
CALL gds.graph.project(
  'Mygraph',
  'Acte',
  'FOLLOWED_BY',
  {
    relationshipProperties: 'Nb_Days'
  }
)
"""
results = db.cypher_query(query)


In [23]:
#run the node2vec algorithm:
query = """
CALL gds.node2vec.stream('Mygraph')
YIELD nodeId, embedding
RETURN nodeId, embedding
"""

from neomodel import db
results = db.cypher_query(query)

In [26]:
results[0][0]

[0,
 [-0.6864916086196899,
  -0.47188785672187805,
  0.29516828060150146,
  0.21024420857429504,
  -0.79278963804245,
  0.1405915915966034,
  -0.14346253871917725,
  0.08222294598817825,
  0.6129434704780579,
  -0.7000496983528137,
  -0.35978636145591736,
  -0.563542366027832,
  -0.062341559678316116,
  -0.11261986941099167,
  0.09980075061321259,
  0.5539767146110535,
  0.5490221381187439,
  0.44031912088394165,
  -1.6658397912979126,
  -0.03663577139377594,
  0.8666197061538696,
  0.11866287887096405,
  0.06265613436698914,
  0.38880953192710876,
  0.4942232370376587,
  -0.2361876666545868,
  -0.7889615893363953,
  -0.17956893146038055,
  -0.15203984081745148,
  0.23860608041286469,
  0.12623882293701172,
  0.01723247393965721,
  0.0513182058930397,
  1.336680293083191,
  -0.00259458739310503,
  0.4711521565914154,
  0.5124085545539856,
  0.6552172303199768,
  -1.2946507930755615,
  0.3633047938346863,
  -0.1660594344139099,
  -0.09053701162338257,
  0.29227375984191895,
  0.03814935

In [19]:
#FIND THE SHORTEST PATH

#CREATE A SHORTEST PATH GRAPH
query = """
CALL gds.graph.project(
    'myGraph2',
    'Acte',
    'FOLLOWED_BY',
    {
        nodeProperties: ['J_Parcours'],
        relationshipProperties: 'Nb_Days'
    }
)
"""
results = db.cypher_query(query)

In [21]:
#RUN THE ALGORITHM

query = """
MATCH (source:Acte {Ref_Acte: 'CS'}), (target:Acte {Phase_Parcours: 'Traitement'})
CALL gds.shortestPath.astar.stream('myGraph2', {
    sourceNode: source,
    targetNode: target,
    JParcours_Property: 'J_Parcours',
    relationshipWeightProperty: 'FOLLOWED_BY'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs,
    nodes(path) as path
ORDER BY index
"""

from neomodel import db
results, meta = db.cypher_query(query, None, resolve_objects=True)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.shortestPath.astar.stream`: Caused by: java.lang.IllegalArgumentException: Multiple errors in configuration arguments:
				No value specified for the mandatory configuration parameter `longitudeProperty`
				No value specified for the mandatory configuration parameter `latitudeProperty`}